### 구글 드라이브 마운트 및 경로 이동

In [ ]:
# 구글 드라이브 마운트
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# 경로 이동
cd ./drive/MyDrive/cakd7/3차프로젝트_dl_1025_1128/5팀\(윤정준,\ 최애림,\ 진청아,\ 이정현a\)/코드

/content/drive/.shortcut-targets-by-id/1SSMjIaDcwniFaLUaXBuU9Ry1IjYbaq-j/cakd7/3차프로젝트_dl_1025_1128/5팀(윤정준, 최애림, 진청아, 이정현a)/코드


# import

In [ ]:
pip install grad-cam

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
from pytorch_grad_cam import GradCAM, HiResCAM, ScoreCAM, GradCAMPlusPlus, AblationCAM, XGradCAM, EigenCAM, FullGrad
from pytorch_grad_cam.utils.model_targets import ClassifierOutputTarget
from pytorch_grad_cam.utils.image import show_cam_on_image
import torchvision
from torchvision.models import resnet18, resnet50, resnet152
import torchvision.transforms as transforms # 이미지 변환 툴
import torch
import cv2
import matplotlib.pyplot as plt
from PIL import Image
import albumentations as A

In [ ]:
# GPU사용 세팅
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

# 이미지 전처리

In [ ]:
# 이미지 tensor화 및 전처리 사용자 함수

def get_img_tensor(img_path):
    img = cv2.imread(img_path)
    h, w, c = img.shape
    print(h, w, c)
    img = Image.fromarray(img[:, :, :3])
    img = torchvision.transforms.ToTensor()(img)
    img = torchvision.transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))(img)
    img = img.unsqueeze(0)
    return img

# 모델 불러오기

In [ ]:
# 모델을 선언
model = resnet50()

# 사전 학습한 모델의 가중치 불러오기
model.load_state_dict(torch.load("./models/best_model.pth",map_location=device))

# CAM을 찍어볼 모델의 특정 레이어를 준비
target_layers = [model.layer4[-1]]

# CAM 함수 선언
cam = XGradCAM(model=model, target_layers=target_layers, use_cuda=False)

# 예측 및 이미지 출력

In [ ]:
img_path = "./train_imgs/BC_01_0531.png"

img = get_img_tensor(img_path)

image = plt.imread(img_path) 

#이미지에 대한 모델 예측값을 저장
pred = int(torch.argmax(torch.nn.Softmax()(model(img))).item()) 

#이미지에 대한 예측값을 target으로 지정
targets = [ClassifierOutputTarget(pred)] 

# CAM을 적용하여 이미지 출력
grayscale_cam = cam(input_tensor=img, targets=targets)
grayscale_cam = grayscale_cam[0, :]
visualization = show_cam_on_image(image, grayscale_cam, use_rgb=True)
plt.imshow(visualization)